# Google Cloud Platform - AutoML tables

https://cloud.google.com/translate/automl/docs/datasets

https://cloud.google.com/automl

https://cloud.google.com/automl-tables/docs/train

https://cloud.google.com/automl-tables/docs

## https://cloud.google.com/automl-tables/docs/notebooks

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/home/neophilex/.ssh/sa-neophilex-alpha-owner.json'

In [ ]:
from google.cloud import automl_v1beta1 as automl
import time

In [ ]:
# TODO(developer): Uncomment and set the following variables

project_id = "neophilex-alpha"
dataset_display_name = "energy_forecasting_solution"
model_display_name='model_iris_001'
path = 'gs://us-central-1x/automl_iris.csv'

client = automl.TablesClient(project=project_id,region='us-central1')

### Execute it only for the first time

In [ ]:
#dataset = client.create_dataset(dataset_display_name)

In [ ]:
dataset = client.get_dataset(project=project_id,
                             region='us-central1',
                             dataset_display_name = dataset_display_name)

In [ ]:
dataset

In [ ]:
response = None
if path.startswith("bq"):
    response = client.import_data(
        dataset_display_name=dataset_display_name, bigquery_input_uri=path
    )
else:
    # Get the multiple Google Cloud Storage URIs.
    input_uris = path.split(",")
    response = client.import_data(
        dataset_display_name=dataset_display_name,
        gcs_input_uris=input_uris,
    )
    
print("Processing import...")
# synchronous check of operation status.
print("Data imported. {}".format(response.result()))

In [ ]:
client.set_target_column(dataset_display_name=dataset_display_name,column_spec_display_name='species')

In [ ]:
training_result = client.create_model(model_display_name='model_iris_001',dataset_display_name=dataset_display_name,
                                      include_column_spec_names=['sepal_length','sepal_width','petal_length','petal_width'],
                                    train_budget_milli_node_hours=1000,
                                      
                   )


print("Training model...")
print("Training operation name: {}".format(training_result.operation.name))
print("Training completed: {}".format(training_result.result()))

In [ ]:
deploy_response = client.deploy_model(model_display_name=model_display_name)


In [ ]:
%%time
deploy_response.result()

In [ ]:
client.predict(inputs={'sepal_length':6.2,
                      'sepal_width':3.4,
                      'petal_length':5.4,
                      'petal_width':2.3},model_display_name=model_display_name)

In [ ]:
undeploy_response = client.undeploy_model(model_display_name=model_display_name)
undeploy_response.result()

In [ ]:
batch_response = client.batch_predict(gcs_input_uris='gs://us-central-1x/automl_batch_predict_iris.csv',
                     model_display_name=model_display_name,
                     gcs_output_uri_prefix='gs://us-central-1x/03_output')

In [ ]:
while batch_response.running():
    time.sleep(10)
    print(batch_response.running())